In [1]:
#Import libraries
import numpy as np
import pandas as pd 
from sklearn.preprocessing import LabelEncoder

In [2]:
#Fetch data
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [3]:
for col in train.columns:
    if train[col].dtype == 'object':
        label = LabelEncoder() 
        label.fit(list(train[col].values) + list(test[col].values)) 
        train[col] = label.transform(list(train[col].values))
        test[col] = label.transform(list(test[col].values))

In [5]:
y_train = train['y'].values
cols = [c for c in train.columns if 'X' in c]
train[cols].dtypes.value_counts()


int64    376
dtype: int64

In [8]:
if train.isnull().any().any():
        print("There are missing values in the dataframe")
else:
    print("There are no missing values in the dataframe")

There are no missing values in the dataframe


In [9]:
if test.isnull().any().any():
        print("There are missing values in the dataframe")
else:
    print("There are no missing values in the dataframe")

There are no missing values in the dataframe


In [10]:
from sklearn.decomposition import PCA
n_comp = 12

In [11]:
# PCA
pca = PCA(n_components=n_comp, random_state=42)
pca2_results_train = pca.fit_transform(train.drop(["y"], axis=1))
pca2_results_test = pca.transform(test)



In [12]:
for i in range(1, n_comp+1):
    train['pca_' + str(i)] = pca2_results_train[:,i-1]
    test['pca_' + str(i)] = pca2_results_test[:, i-1]
    

y_train = train["y"]
y_mean = np.mean(y_train)



In [16]:
import xgboost as xgb

# Parameters
xgb_params = {
    'n_trees': 500, 
    'eta': 0.02,
    'max_depth': 4,
    'objective': 'reg:linear'
}

d_train = xgb.DMatrix(train.drop('y', axis=1), y_train)
d_test = xgb.DMatrix(test)

C:\Users\dishpate\AppData\Local\Continuum\anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\Users\dishpate\AppData\Local\Continuum\anaconda3\lib\site-packages\xgboost\core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \


In [17]:

num_boost_rounds = 150
# model training
model = xgb.train(dict(xgb_params, silent=0), d_train, num_boost_round=num_boost_rounds)


[21:44:34] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


In [18]:
from sklearn.metrics import r2_score
print(r2_score(model.predict(d_train), d_train.get_label()))

# make predictions and save results
y_pred = model.predict(d_test)

-0.10143047742627709


In [19]:
output = pd.DataFrame({'id': test['ID'].astype(np.int32), 'y': y_pred})
output.to_csv('sub.csv', index=False)